In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_images, plot_heatmap

In [ ]:
sdf_calls_path = os.path.join("benchmarks", "performance")

In [ ]:
benchmark_filenames = {}

for dir_name in os.listdir(sdf_calls_path):
    sub_dir_path = os.path.join(sdf_calls_path, dir_name)
    if (os.path.isdir(sub_dir_path)):
        benchmark_filenames[dir_name] = os.listdir(sub_dir_path)

In [ ]:
benchmark_images = {}
time_meassurments = {}

for method, filenames in benchmark_filenames.items():
    folder_path = os.path.join(sdf_calls_path, method)

    benchmark_images[method] = []
    for filename in filenames:
        if (filename.split(".")[-1] == "png"):
            benchmark_file_path = os.path.join(folder_path, filename)
            benchmark_image_bgr = cv2.imread(benchmark_file_path)
            max_iteration_count = int((os.path.basename(benchmark_file_path).split(".")[-2]).split("_")[-1])
            benchmark_images[method].append((max_iteration_count, benchmark_image_bgr))
        elif (filename.split(".")[-1] == "txt"):
            time_meassurment_path = os.path.join(folder_path, filename)
            time_meassurment_file = open(time_meassurment_path, "r+")
            mm = time_meassurment_file.readline().split(" ")
            time_meassurment_file.close()
            time_meassurments[method] = (float(mm[0]), float(mm[1]))

    benchmark_images[method].sort(key=lambda x: x[0])
        
benchmark_images = dict(sorted(benchmark_images.items(), key=lambda x: x[0])[::-1])
time_meassurments = dict(sorted(time_meassurments.items(), key=lambda x: x[0])[::-1])

In [ ]:
benchmark_sdf_call_count_images_gray = {}
avg_sdf_counts_per_pixel = {}
max_iter_counts = set()

for y, (method, images) in enumerate(benchmark_images.items()):
    benchmark_sdf_call_count_images_gray[method] = []
    avg_sdf_counts_per_pixel[method] = []

    for x, (max_iteration_count, image_bgr) in enumerate(images):
        image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
        
        sdf_count_sum = int((cv2.sumElems(image_gray)[0] / 255.0) * max_iteration_count)
        avg_sdf_count_per_pixel = sdf_count_sum / (image_gray.shape[0] * image_gray.shape[1])

        benchmark_sdf_call_count_images_gray[method].append(image_gray)
        avg_sdf_counts_per_pixel[method].append(round(avg_sdf_count_per_pixel, 3))
        max_iter_counts.add(max_iteration_count)

max_iter_counts = sorted(max_iter_counts)

In [ ]:
plot_images(
    np.array(list(benchmark_sdf_call_count_images_gray.values())),
    [" ".join(str(i).split("_")) for i in list(benchmark_sdf_call_count_images_gray.keys())],
    [str(i) for i in list(max_iter_counts)],
    50,
    35,
    "SDF call counts (divided by max iteration counts) of different tracing methods at different max iteration counts"
)

In [ ]:
plot_heatmap(
    np.array(list(avg_sdf_counts_per_pixel.values())),
    [" ".join(str(i).split("_")) for i in list(avg_sdf_counts_per_pixel.keys())],
    [str(i) for i in list(max_iter_counts)],
    "",
    10,
    6,
    "Max iteration count",
    "Tracing method",
    "Average sdf call count per pixel"
)

In [ ]:
plot_heatmap(
    np.array(list(time_meassurments.values())),
    [" ".join(str(i).split("_")) for i in list(time_meassurments.keys())],
    ["chrono", "gl querry"],
    "ms",
    5,
    6,
    "Meassurment method",
    "Tracing method",
    "Averaged render times in ms"
)